# Piano to Sheet

Converting .wav piano pieces into music sheets.

I

## Pipeline break down ( beta)

1. Loading in the wave file,
2. 



## Audio Processing  Libraries

In [ ]:
pip install torch torchvision torchaudio

For the processing libraries, we will be using pretty_midi to extract out label information from the given midi files.  
And for processing our wav audio files. We will be using librosa.

## Machine Learning Lib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import librosa 
import json
import csv
import pandas as pd
import pretty_midi as pm
import mido
import IPython.display as ipd
import torch
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import math

# Model Saving funtion and Model loading function

Having a model saving function and a modle loading function in which we can train the model in small epoch progress.

# Preprocessing the data

In [ ]:

labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
with open(labels_file_path, 'r') as file:
    data = json.load(file)

all_sets = {}
all_sets['train'] = []
all_sets['validation'] = []
all_sets['test'] = []

def sortingsets (data, allsets):
    for key in data:
        if data[key] == 'train':
            all_sets['train'].append(key)
        elif data[key] == 'validation':
            all_sets['validation'].append(key)
        else:
            all_sets['test'].append(key)

def save_index_to_csv(all_sets):
    for key in all_sets:
    
        path = f"traindata/maestro-v3.0.0-midi/maestro-v3.0.0/{key}_indicies.csv"
        df = pd.DataFrame({'Index': all_sets[key]})
        df.to_csv(path, index=False)

def load_index_from_csv(path):
    indices = []
    with open(path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            indices.append(int(row[0]))
    return indices





traindata/maestro-v3.0.0-midi/maestro-v3.0.0/train_indicies.csv  
traindata/maestro-v3.0.0-midi/maestro-v3.0.0/test_indicies.csv   
traindata/maestro-v3.0.0-midi/maestro-v3.0.0/validation_indicies.csv

In [ ]:
#save_index_to_csv(all_sets)

### Data Preprocessing
implenting utility functions such as the randomizing the data set for training.

In [ ]:
### making the random seed 
np.random.seed(40)

In [ ]:
## Randomizing the data set index for training purposes
def randomizeing(data_set):
    ds = np.array(data_set)
    np.random.shuffle(ds)
    return ds

## Select n indices from the givien data set
def select_N_randomized_from_set(n, data_set):
    nparry = randomizeing(data_set)
    return nparry[:n]


## ----- ----- ---------- loading function ------------------------------ ##

## function loading in the wav function
def load_wav_from_index(index):
    labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
    with open(labels_file_path, 'r') as file:
        data = json.load(file)
    wav_path = "traindata/maestro-v3.0.0/maestro-v3.0.0/" + data['audio_filename'][str(index)]
    
    return librosa.load(wav_path, sr=None)

## function loading in the midi function
def load_midi_from_index(index):
    labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
    with open(labels_file_path, 'r') as file:
        data = json.load(file)
    midi_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/" + data['midi_filename'][str(index)]
    return pm.PrettyMIDI(midi_path)

## ----- ----- -------- Path showing function-------------------------------- ##
    
## showing the file path audio of the wave
def show_wav_path(index):
    labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
    with open(labels_file_path, 'r') as file:
        data = json.load(file)
    wav_path = "traindata/maestro-v3.0.0/maestro-v3.0.0/" + data['audio_filename'][str(index)]
    return wav_path

## Showing the file path of the midi file of data[index]
def show_midi_path(index):
    labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
    with open(labels_file_path, 'r') as file:
        data = json.load(file)
    midi_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/" + data['midi_filename'][str(index)]
    return midi_path

## loading in the wav and midi pair
def load_wav_midi_pair(index): ## (wav, midi)
    return load_wav_from_index(index), load_midi_from_index(index)

In [ ]:
train_data = load_index_from_csv("traindata/maestro-v3.0.0-midi/maestro-v3.0.0/train_indicies.csv")
print(len(train_data))
subset_train_data = select_N_randomized_from_set(50, train_data)

print(len(subset_train_data))

# AUDIO EXPERIMENTATION

In [ ]:
(wav, sr), midi = load_wav_midi_pair(1025)
# midi = load_midi_from_index(1025)


In [ ]:
ballade1, sr = load_wav_from_index(505)

noised_ballade1 = add_gaussian_noise(ballade1, noise_level=0.01)

In [ ]:
ipd.Audio(data= ballade1, rate= sr)

### Audio Preprocessing functions

#### Pretty_midi note
The MIDI object is used from the python package pretty_midi.  
Using pretty_midi range from 0 to 127. we can later tranform this into the range of 0 to 87 to match a piano

In [206]:



""" This function extracts all played notes in the midi Object, which it will be futher trained with the aligne ed wave object
    input: pm object
    It is good for debugging and seeing the midi object
"""
def extract_midi_notes(midi):
    notes = []
    for instrument in midi.instruments:
        for note in instrument.notes:
            notes.append({
                'pitch': note.pitch,
                'start': note.start,
                'end': note.end,
                'velocity': note.velocity
            })
    #preprocessing the sort
    notes.sort(key=(lambda x: x['start']))
    return notes




""" Wrting a peekable Generator for midi object"""
class PeekableGenerator:
    def __init__(self, generator):
        self._generator = generator
        self._next_item = None
        self._has_next = False
        self._advance()

    def _advance(self):
        try:
            self._next_item = self._generator.__next__()
            self._has_next = True
        except StopIteration:
            self._next_item = None
            self._has_next = False

    def peek(self):
        if not self._has_next:
            raise StopIteration("No more elements to peek at.")
        return self._next_item

    def __next__(self):
        if not self._has_next:
            raise StopIteration("No more elements.")
        current = self._next_item
        self._advance()
        return current

    def has_next(self):
        return self._has_next

    def __iter__(self):
        yield self._next_item
        self._advance()

    

"""Generator to yield midi note object at the frame during the classification
    input: pm object
"""
def midi_yielding(midi):
    all_midi_obj :list = extract_midi_notes(midi)
    ##Processing
    for note in all_midi_obj:
        yield note


""" Yielding a list of midi notes information where it fits the time frame automatically.
    Implemented using overlapping frame structure for the training.
    Begin at 0, the frame jumping at the speed of jump_len, the size of the frame is frame_len
    This function will yield the frame at the given parameter.
"""
def frame_aligning_midi(t: int, jump_len:int, frame_len: int, sr:int, midi):
    midi_generator = PeekableGenerator(midi_yielding(midi))
    midi_labels = []

    jump_time_fraction: float = jump_len * (1/sr)
    frame_time_fraction: float = frame_len * (1/sr)
    print(f"The frame jumping rate is at {jump_time_fraction} seconds, and the size of the frame is at {frame_time_fraction}.")
    framing = [t*jump_time_fraction, t*jump_time_fraction + frame_time_fraction]
    
    last_note = 0
    while midi_generator.has_next() or last_note > framing[0]:
        
        while midi_generator.has_next() and midi_generator.peek()['start'] >= framing[0] and midi_generator.peek()['start'] < framing[1]:
            try:
                midi_labels.append(midi_generator.__next__())
            except StopIteration:
                print("Generator exhausted, no Midi Objectis being added to the label")
                break
        
        ## Yielding the list of midi notes that are fitted in side the frame
        yield midi_labels

        midi_labels.sort(key=(lambda x: x['end']))
        ##calculating the next frame time step and removing the items from the previous frame
        framing [0] += jump_time_fraction
        framing [1] += jump_time_fraction
        while len(midi_labels) > 0 and midi_labels[0]['end'] < framing[0]:
            midi_labels.pop(0)
        
        if len(midi_labels) > 0 and midi_labels[0]['end'] > framing[0]:
            last_note = midi_labels[0]['end']

    
        
  



    
""" This function returns the aligned frame at the wav data,"""
def frame_aligning_wav(t: int, jump_len: int, frame_len: int, wav):
    begin = t * jump_len
    return wav[begin: begin + frame_len]


""" This function returns the amount seconds of audio data from the wav, began on t, while using  """

def audio_segment_of(t: int, wav, seconds: float, sr: int, jump_len: int = 512, frame_len: int = 2048, ):
    size = int(seconds*sr)
    begin = t*jump_len
    
    return wav[begin: begin + size]

def audio_segment_between(begin, end, wav, sr):
    return wav[int(begin*sr): int(end*sr)]

    


### Constructing Mel Spectrogram From wav frames

In [ ]:
(ballade1, sr), midi = load_wav_midi_pair(505)

for i in frame_aligning_midi(0, 512, 2048, sr, midi):
    print(i)

In [ ]:

ballade1, sr = load_wav_from_index(505)
audio = audio_segment_of(0, ballade1, 3, sr)
audio1 = audio_segment_between(500,501,ballade1, sr)


In [ ]:
ipd.Audio(data= audio1, rate= sr)

In [ ]:
def extract_mel_spectrogram(audio, sr, n_mels=88, hop_length=512, n_fft=2048): ##_fft is the frame length 
    """
    Extract a mel-spectrogram from raw audio.
    """
    mel_spec = librosa.feature.melspectrogram(
        y=audio, sr=sr, n_mels=n_mels, hop_length=hop_length, n_fft=n_fft
    )
    # Convert to log scale
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db

# Example audio input
mel_spectrogram = extract_mel_spectrogram(audio1, sr)

# Normalize the spectrogram for display
mel_spectrogram_normalized = (mel_spectrogram - mel_spectrogram.min()) / (mel_spectrogram.max() - mel_spectrogram.min())

# Plot the mel-spectrogram
plt.figure(figsize=(10, 4))
librosa.display.specshow(mel_spectrogram_normalized, sr=sr, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-Spectrogram')
plt.show()


In [ ]:
mel_spectrogram.shape


### Calculating the added noise level to the wav function.

In [ ]:
def compute_noise_power(audio, snr_dB):
    """
    Compute the noise power needed for a given SNR in dB.
    :param audio: NumPy array of the audio signal.
    :param snr_dB: Desired Signal-to-Noise Ratio in dB.
    :return: Noise power.
    """
    # Calculate signal power (mean squared amplitude)
    signal_power = np.mean(audio ** 2)
    
    # Convert SNR from dB to linear scale
    snr_linear = 10 ** (snr_dB / 10)
    
    # Calculate noise power
    noise_power = signal_power / snr_linear
    return noise_power

div = 100
somewav = select_N_randomized_from_set(div, train_data)
print(len(somewav))
summing = 0 #summing the proper noise level
for i in range(10):
    wav, _ = load_wav_from_index(somewav[i])
    summing += np.sqrt(compute_noise_power(wav, 20)) ## desired training SNR to be 15 dB

print(f"Our desired training noise is about {summing/div}, setting it to be our default added noise to wav")



In [ ]:
def add_gaussian_noise(audio, noise_level=0.0006):
    
    noise = np.random.normal(0, noise_level, audio.shape)
    return audio + noise

def framelining (time, jump, sr):
    
    frametime = jump/sr
    print(22*frametime)

framelining(1, 2048, 44100)

### Training functions

In [218]:

"""This function generates the matrix label to aligne the mal_spectral gram
each matrix should consired num_frame,
poping fram_jumping after the matrix is being yield

jumping time should be
frame_jumping * (frame_len /sr)
"""



def label_generator(num_frame, frame_jumping, jump_len:int, frame_len: int, sr:int, midi):
    midi_frame_gen = PeekableGenerator(frame_aligning_midi(0, frame_len, frame_len, sr, midi)) ## Note, while using the architechture of mel_spectrogram, the we don't need to consider the jump offset in the midi side
    label = []
    counter = 0
    
    
    concur_time = 0
    concur_time_end = 0
    frame_time = frame_len /sr

    
    while midi_frame_gen.has_next():
        if (len(label) < num_frame):
            label.append(list(midi_frame_gen.__next__()))
            concur_time_end += frame_time
            
        else:
            counter+=1
            yield label
            print(f"label generator output is {concur_time}, {concur_time_end}")
           
        
            for i in range(int(frame_jumping)):
                label.pop(0)
                concur_time += frame_time
                


"""This function tries to mimic the decayed velocity miniking the sound at which a piano has been decayed"""
def velocity_decay_sustain (velocity, onset, at_time):
    if (at_time - onset < 0.2):
        return velocity
    else:
        return math.exp((onset - at_time) * 0.6) * velocity


"""formating the label of list, in to a matrix of 88 * 3 matrix.
    each row represent a strikable key, 
    column 1 (being stricked) : 0, 1 (classification purpose)
    column 2 (onset timer) : the set of positive interger that is less than onset. (Regression purpose)
    column 3 (velocity of which is being stricked) : the set of positive integer that is less than onset. (Regression purpose) 
        note for the velocity of the piano key will be approximately alingned with a decay parameter"""
def label_formater(label, frameonset):
    ret_label = np.zeros((88,3))
    
    print(label)
    for frame in label :
        for key_obj in frame:
            pitch = key_obj['pitch'] - 20
            ret_label[pitch][0] = 1
            ret_label[pitch][1] = key_obj['start'] if key_obj['start'] > frameonset else frameonset
            ret_label[pitch][2] = velocity_decay_sustain(key_obj['velocity'],  key_obj['start'], frameonset) 

    return ret_label




"""This function assure the input for the training will retain the dimension in the case of track is ending."""
def construct_input(spectrogram, x, y):
    if spectrogram.shape == (x,y):
        return spectrogram
    else: 
        ret = np.zeros((x,y))
        for i in range(spectrogram.shape[0]):
            ret[i] = spectrogram[i]
        return ret

def train_segment(wav, sr, length=1, hop=0.5):
    ret = []
    max = len(wav)/sr
    begin = 0
    
    print((begin < max))
    while (begin < max):
        adding = length if begin + length < max else max - begin
        ret.append((begin, begin + adding))
        begin += hop

    return ret
    

"""The training is based on how many frame should be trained at a time,
    the default setting is suited for the expriment set up above,
    num_frame is tried to aligned it to approx 1 second of the sample
    segment_jumping would be trying to get 50% of the sample audio
"""
def training(model, index, num_frame=22, segment_jump = 0.5, frame_length = 2048, hop_length = 512):
    #load model 


    # fetching infomation
    (wav, sr), midi = load_wav_midi_pair(index)
    
    frame_time =  frame_length/sr
    quick_sample = audio_segment_between(0, num_frame*frame_time, wav, sr)
    x, y = extract_mel_spectrogram(quick_sample, sr).shape
    segments = train_segment(wav, sr, length = num_frame * frame_time, hop= num_frame * frame_time/2)
    print(segments[0])
    label_gen = PeekableGenerator(label_generator(num_frame, num_frame*segment_jump, hop_length, frame_length, sr, midi))
    
    for beg, end in segments:
        
        if (label_gen.has_next() is False) :
            break

        audio = audio_segment_between(beg, end, wav, sr)
        mel_spectrogram = extract_mel_spectrogram(audio, sr, hop_length= hop_length, n_fft=frame_length)
        input_data = construct_input(mel_spectrogram, x,y)
        label = label_formater(label_gen.__next__(), beg)
        print(f"input data shape{input_data.shape}")
        print(f"label's shape{label.shape}")
        break
    
    #save model 



def partial_training(model, time_begin, time_end, wav, sr, midi):
    return 0
   



In [219]:
training(123, 505)

True
(0, 1.0216780045351475)
The frame jumping rate is at 0.046439909297052155 seconds, and the size of the frame is at 0.046439909297052155.
label generator output is 0, 1.021678004535147
[[], [], [], [], [], [], [], [], [], [{'pitch': 48, 'start': np.float64(0.9854166666666666), 'end': np.float64(1.809375), 'velocity': 87}, {'pitch': 36, 'start': np.float64(0.9874999999999999), 'end': np.float64(1.8968749999999999), 'velocity': 83}], [{'pitch': 48, 'start': np.float64(0.9854166666666666), 'end': np.float64(1.809375), 'velocity': 87}, {'pitch': 36, 'start': np.float64(0.9874999999999999), 'end': np.float64(1.8968749999999999), 'velocity': 83}], [{'pitch': 48, 'start': np.float64(0.9854166666666666), 'end': np.float64(1.809375), 'velocity': 87}, {'pitch': 36, 'start': np.float64(0.9874999999999999), 'end': np.float64(1.8968749999999999), 'velocity': 83}], [{'pitch': 48, 'start': np.float64(0.9854166666666666), 'end': np.float64(1.809375), 'velocity': 87}, {'pitch': 36, 'start': np.floa

In [ ]:
label_gen = PeekableGenerator(label_generator(22, 22*0.5, 512, 2048, sr, midi))
(ballade1, sr), midi= load_wav_midi_pair(505)
audio = audio_segment_of(0, ballade1, 3, sr)
audio1 = audio_segment_between(500,501,ballade1, sr)
segm = train_segment(ballade1, sr, length = 22* 2048/sr, hop= 22* 2048/sr/2)
print(len(segm))
print(segm[-1])
print(segm[-2])




In [ ]:
counter = 0
while label_gen.has_next() or counter < len(segm):
    if (counter < len(segm)):
        print(f"time segment function o  {segm[counter]}")
        counter+=1
    if (label_gen.has_next()):
        print(label_gen.__next__())
    

    


In [ ]:

## num_frame=22, segment_jump = 0.5, frame_length = 2048, hop_length = 512
(ballade1, sr), midi= load_wav_midi_pair(505)
audio = audio_segment_of(0, ballade1, 3, sr)
audio1 = audio_segment_between(500,501,ballade1, sr)
segm = train_segment(ballade1, sr, length = 22* 2048/sr, hop= 22* 2048/sr/2)
print(len(segm))

In [ ]:
print(len(ballade1)/sr)
print(sr)